<a href="https://colab.research.google.com/github/xarvel/DataScience/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf

import numpy as np
import os
import time

In [24]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [76]:
def text_stats(text):
  # length of text is the number of characters in it
  print(f'Length of text: {len(text)} characters')
  # The unique characters in the file
  vocab = sorted(set(text))
  print(f'{len(vocab)} unique characters')

  return vocab

In [77]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = text_stats(text)

Length of text: 1115394 characters
65 unique characters


In [78]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [79]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [29]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

In [30]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [31]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

In [32]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [33]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [34]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [35]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [36]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [37]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [38]:
seq_length = 100

In [39]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [40]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [41]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [42]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [43]:
dataset = sequences.map(split_input_target)

In [44]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [45]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [46]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [47]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [48]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [49]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [50]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [52]:
sampled_indices

array([17, 55, 30, 48,  9, 51,  6, 34, 15, 24, 32, 31, 54, 46, 36, 16,  5,
        6, 64,  1, 30, 57, 29, 45, 61, 27, 62, 53,  0, 26, 26, 53, 35, 55,
       58, 62, 42, 26, 34, 61, 22, 49, 36,  8, 63, 23, 38,  3, 11, 29,  5,
       41, 21, 30,  6, 27, 34, 11, 56, 32, 14, 28, 53,  3, 56, 24, 51, 21,
       33, 11, 47, 19, 49, 61, 18, 50, 52, 45,  7, 63, 38, 47, 52, 24,  7,
       57, 18, 16, 16, 12, 45, 27,  4, 26, 19, 44,  6, 30, 42, 57])

In [53]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'She embraces him.\n\nCAMILLO:\nShe hangs about his neck:\nIf she pertain to life let her speak too.\n\nPOL'

Next Char Predictions:
 b"DpQi.l'UBKSRogWC&'y\nQrPfvNwn[UNK]MMnVpswcMUvIjW-xJY!:P&bHQ'NU:qSAOn!qKlHT:hFjvEkmf,xYhmK,rECC;fN$MFe'Qcr"


In [54]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [55]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1910553, shape=(), dtype=float32)


In [56]:
tf.exp(example_batch_mean_loss).numpy()

66.0925

In [57]:
model.compile(optimizer='adam', loss=loss)

In [58]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [59]:
EPOCHS = 20

In [60]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 15s 60ms/step - loss: 2.7266
Epoch 2/20
172/172 [==============================] - 11s 54ms/step - loss: 1.9914
Epoch 3/20
172/172 [==============================] - 11s 55ms/step - loss: 1.7138
Epoch 4/20
172/172 [==============================] - 12s 59ms/step - loss: 1.5534
Epoch 5/20
172/172 [==============================] - 11s 57ms/step - loss: 1.4540
Epoch 6/20
172/172 [==============================] - 12s 59ms/step - loss: 1.3866
Epoch 7/20
172/172 [==============================] - 12s 61ms/step - loss: 1.3327
Epoch 8/20
172/172 [==============================] - 12s 61ms/step - loss: 1.2877
Epoch 9/20
172/172 [==============================] - 12s 61ms/step - loss: 1.2456
Epoch 10/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2047
Epoch 11/20
172/172 [==============================] - 12s 59ms/step - loss: 1.1654
Epoch 12/20
172/172 [==============================] - 12s 61ms/step - loss: 1.1239
E

In [61]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [62]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [63]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
They have been four-bledged tine: and there's the boy ornament:
And let their demand post thou no others
To the madam. Honest upper me far further,
Instinging pity dignity: he smears him back again.

KING RICHARD III:
How now, charged me, boy!

Messenger:
My gracious up, Hastings and three.

KING RICHARD III:
Say that I say; bit it be there.

BUCKINGHAM:
How comes the minister nack, what lesson's members
could have dones conceuns to die yet betime.

DOKE OLBHARD:
And to-morrow art thou now? this honour hath no hopeo's
ratel guide their sure did living;
And when he slain, whose public heart's endmance,
And that stabs temples of this virtuous tribunes
In those fruitful Loncoman to conceal our voices;
Stand you thy wall.

Citizens:
These news are wormest, allow your hinder passage.

KING RICHARD II:
Mine ear I shall not be commanded for
some powerful kinsman come; or if our grief hath
Been poor breasty forgivence.

YORK:
Here comes the kingdom of peace but unstand.

ROMEO:
Am I now

In [64]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nShe speaks:\nO, miserable my lord, gone to this end.\n\nKING EDWARD IV:\nWhy, 'tis good neighbour, and, with my hand\nHath held the fiftered to her friends.\n\nGONZALO:\nHere, Hortensio.\n\nBoth\nTribunes: what news? what, Johe not? Shall we go?\n\nROMEO:\nWhy, sir, what you have wont? Thy love did vasient; that we maid\nprepared this small be provised prince, Henry's mejicians,\nTake nothing or forbid: it was he wear their heat, in him\nTo enter in the faults which I have done.\nGo,--your Romeo will be manied to mock no other of curses;\nOf whether like next buits with paintenance,\nMore than the moveth state that spoils where he\nWas this, to dispose the soldiers to the ground,\nAnd chat the mouth of more than mistresse o'er his meed:\nNot shunned the harmy stain'd about his sacred man,\nWhich way, until thou been mile!\nHis injuries shall persuade him, nor he, or grace sir:\nI'll tell you where he should be much, but to conserve\nand true misation; and, thou vo

In [65]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [66]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The field trust forgeth thee on thy wildness,
And where he chiefs, shame thy affair.

MONSA:
I memp


In [67]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [68]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [69]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [70]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 59ms/step - loss: 2.7044


In [71]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1511
Epoch 1 Batch 50 Loss 2.0434
Epoch 1 Batch 100 Loss 1.9218
Epoch 1 Batch 150 Loss 1.8372

Epoch 1 Loss: 1.9793
Time taken for 1 epoch 13.63 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8276
Epoch 2 Batch 50 Loss 1.7126
Epoch 2 Batch 100 Loss 1.7073
Epoch 2 Batch 150 Loss 1.6494

Epoch 2 Loss: 1.7042
Time taken for 1 epoch 12.20 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6035
Epoch 3 Batch 50 Loss 1.5375
Epoch 3 Batch 100 Loss 1.5374
Epoch 3 Batch 150 Loss 1.4967

Epoch 3 Loss: 1.5444
Time taken for 1 epoch 12.10 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.5260
Epoch 4 Batch 50 Loss 1.4214
Epoch 4 Batch 100 Loss 1.4348
Epoch 4 Batch 150 Loss 1.4225

Epoch 4 Loss: 1.4451
Time taken for 1 epoch 12.18 sec
_____________________________________________________________________